In [1]:
# Установка transformers
!pip install transformers


[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Установка catboost
!pip install catboost


[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from itertools import groupby

In [4]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from catboost import CatBoostClassifier, Pool
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [5]:
#Считывание данных в DataFrame 


train= pd.read_csv('train_dataset_train.csv', sep=',', index_col=None)
test= pd.read_csv('test_dataset_test.csv', sep=',', index_col=None)

In [6]:
train

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,nd_mean_2021-05-17,nd_mean_2021-05-21,nd_mean_2021-05-26,nd_mean_2021-05-27,nd_mean_2021-05-29,nd_mean_2021-06-06,nd_mean_2021-06-09,nd_mean_2021-06-13,nd_mean_2021-06-16,nd_mean_2021-06-18,nd_mean_2021-06-19,nd_mean_2021-06-20,nd_mean_2021-06-22,nd_mean_2021-06-25,nd_mean_2021-06-27,nd_mean_2021-07-07,nd_mean_2021-07-08,nd_mean_2021-07-09,nd_mean_2021-07-13,nd_mean_2021-07-15,nd_mean_2021-07-17,nd_mean_2021-07-20,nd_mean_2021-07-26,nd_mean_2021-07-27,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-07,nd_mean_2021-08-12,.geo,nd_mean_2021-04-15,nd_mean_2021-04-18,nd_mean_2021-04-25,nd_mean_2021-04-27,nd_mean_2021-04-29,nd_mean_2021-04-30,nd_mean_2021-06-07,nd_mean_2021-06-12,nd_mean_2021-06-28,nd_mean_2021-08-01,nd_mean_2021-08-11,nd_mean_2021-05-01,nd_mean_2021-05-03,nd_mean_2021-05-10,nd_mean_2021-05-15,nd_mean_2021-06-02,nd_mean_2021-06-03,nd_mean_2021-07-04,nd_mean_2021-08-10,nd_mean_2021-08-23,nd_mean_2021-04-20,nd_mean_2021-04-23,nd_mean_2021-05-09,nd_mean_2021-05-19,nd_mean_2021-05-20,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop
0,3536,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,0.484102,0.201013,0.029708,0.397833,0.384805,0.015936,0.007275,0.441994,0.172777,0.000000,0.412072,0.507029,0.490741,0.549791,0.460996,0.100555,0.706968,0.691171,0.711135,0.731325,0.718093,0.000000,0.147169,0.728053,0.112933,0.727516,0.000000,0.000000,0.658016,"{""type"":""Polygon"",""coordinates"":[[[42.49107269...",0.187150,0.262300,0.077577,0.000476,0.000000,0.000000,0.111821,0.038902,0.155038,0.754859,0.000000,0.121799,0.083970,0.213006,0.358304,0.271128,0.173833,0.014060,0.030883,0.034858,0.063134,0.000000,0.102757,0.425571,0.108999,0.118854,0.000000,0.000000,0.026784,0.126832,0.614770,0.008857,0.081498,3
1,3739,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,0.519456,0.501486,0.047332,0.448796,0.692873,0.628463,0.016747,0.000000,0.059826,0.000000,0.492250,0.708562,0.668011,0.675501,0.694017,0.654727,0.709037,0.696734,0.268523,0.076741,0.638567,0.676917,0.460940,0.059759,0.073804,0.666918,0.089721,0.184231,0.601685,"{""type"":""Polygon"",""coordinates"":[[[50.25545110...",0.245285,0.349133,0.173578,0.196361,0.015048,0.242130,0.708402,0.150383,0.432990,0.706226,0.618475,0.047408,0.000000,0.160796,0.031104,0.000000,0.040128,0.194295,0.062582,0.000000,0.064671,0.024539,0.000000,0.248935,0.134469,0.208268,0.000000,0.000000,0.111148,0.174914,0.179612,0.113071,0.046997,4
2,1294,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,0.000000,0.561541,0.000000,0.029533,0.245910,0.650065,0.000000,0.516493,0.000000,0.717313,0.649345,0.740904,0.652131,0.739836,0.595076,0.741171,0.648106,0.241615,0.586071,0.551204,0.522871,0.543462,0.000000,0.172532,0.119380,0.321827,0.411168,0.357296,0.000000,"{""type"":""Polygon"",""coordinates"":[[[37.99698998...",0.006391,0.000000,0.041751,0.169020,0.011260,0.118255,0.056252,0.011727,0.172335,0.000000,0.172080,0.230169,0.024845,0.151504,0.002801,0.011655,0.000000,0.135650,0.045370,0.000000,0.000000,0.112471,0.028961,0.127633,0.084467,0.000000,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,2
3,2859,19,0.105976,0.052500,0.001917,0.011531,0.042316,0.000000,0.000000,0.043604,0.510271,0.511552,0.000000,0.524763,0.707093,0.645025,0.753284,0.000000,0.032089,0.064643,0.543500,0.706039,0.617195,0.712928,0.628014,0.563310,0.591204,0.557154,0.188446,0.466411,0.161966,0.425800,0.094829,0.474245,0.242224,0.000000,0.055983,0.060656,0.327025,"{""type"":""Polygon"",""coordinates"":[[[44.72154181...",0.293425,0.318047,0.072589,0.000000,0.145339,0.007611,0.008742,0.099960,0.229617,0.152193,0.379196,0.046483,0.063252,0.099087,0.412416,0.441948,0.7471

In [7]:
test

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,nd_mean_2021-05-17,nd_mean_2021-05-21,nd_mean_2021-05-26,nd_mean_2021-05-27,nd_mean_2021-05-29,nd_mean_2021-06-06,nd_mean_2021-06-09,nd_mean_2021-06-13,nd_mean_2021-06-16,nd_mean_2021-06-18,nd_mean_2021-06-19,nd_mean_2021-06-20,nd_mean_2021-06-22,nd_mean_2021-06-25,nd_mean_2021-06-27,nd_mean_2021-07-07,nd_mean_2021-07-08,nd_mean_2021-07-09,nd_mean_2021-07-13,nd_mean_2021-07-15,nd_mean_2021-07-17,nd_mean_2021-07-20,nd_mean_2021-07-26,nd_mean_2021-07-27,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-07,nd_mean_2021-08-12,.geo,nd_mean_2021-04-15,nd_mean_2021-04-18,nd_mean_2021-04-25,nd_mean_2021-04-27,nd_mean_2021-04-29,nd_mean_2021-04-30,nd_mean_2021-06-07,nd_mean_2021-06-12,nd_mean_2021-06-28,nd_mean_2021-08-01,nd_mean_2021-08-11,nd_mean_2021-05-01,nd_mean_2021-05-03,nd_mean_2021-05-10,nd_mean_2021-05-15,nd_mean_2021-06-02,nd_mean_2021-06-03,nd_mean_2021-07-04,nd_mean_2021-08-10,nd_mean_2021-08-23,nd_mean_2021-04-20,nd_mean_2021-04-23,nd_mean_2021-05-09,nd_mean_2021-05-19,nd_mean_2021-05-20,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24
0,611,26,0.196691,0.000000,0.259303,0.052522,0.133353,0.000000,0.058353,0.273228,0.944027,0.078499,0.128236,0.800393,0.079663,0.140596,0.029266,0.000000,0.021042,0.719716,0.781434,0.787326,0.788925,0.826322,0.779488,0.157232,0.796109,0.807952,0.722456,0.751955,0.263868,0.086229,0.000000,0.743621,0.068059,0.719171,0.000000,0.000000,0.164840,"{""type"":""Polygon"",""coordinates"":[[[35.96545926...",0.083038,0.244827,0.066446,0.153113,0.101380,0.136574,0.000000,0.000000,0.051334,0.593911,0.192016,0.075291,0.269741,0.552214,0.601965,0.057177,0.194608,0.188294,0.000000,0.065030,0.241219,0.064947,0.099994,0.030786,0.159111,0.127732,0.188092,0.000000,0.200005,0.714292,0.084197,0.157733,0.197326
1,6417,98,0.091192,0.000000,0.125823,0.000000,0.027636,0.433244,0.344155,0.461029,0.429587,0.262036,0.170126,0.557339,0.547728,0.586947,-0.000724,0.819140,0.541147,0.113049,0.869505,0.905817,0.871477,0.894545,0.799347,0.851159,0.800004,0.827959,0.752727,0.658426,0.567668,0.494359,0.417848,0.347613,0.476544,0.317670,0.329803,0.007428,0.167479,"{""type"":""Polygon"",""coordinates"":[[[39.50520518...",0.122470,0.241003,0.081871,0.183747,0.000000,0.104354,0.187874,0.108817,0.059370,0.000000,0.106484,0.144733,0.037462,0.000000,0.136006,0.051131,0.004498,0.209801,0.000000,0.000000,0.181600,0.108175,0.248783,0.064743,0.000000,0.039850,0.067910,0.084682,0.025694,0.000000,0.108945,0.141947,0.000000
2,3352,21,0.000000,0.033368,0.285057,0.028275,0.089219,0.080082,0.220537,0.154620,0.612726,0.000000,0.145156,0.652396,0.677490,0.013524,0.025940,0.431207,0.000000,0.217142,0.744129,0.788293,0.034371,0.741114,0.716090,0.051522,0.636270,0.604504,0.672238,0.618853,0.537735,0.000000,0.213445,0.646816,0.030772,0.534127,0.000000,0.000000,0.477381,"{""type"":""Polygon"",""coordinates"":[[[39.30105409...",0.259347,0.297489,0.000000,0.261710,0.327636,0.169726,0.122756,0.038436,0.651745,0.549128,0.469352,0.054238,0.096321,0.043020,0.502996,0.112645,0.240370,0.220739,0.026141,0.000000,0.000000,0.374026,0.037591,0.129930,0.230840,0.034613,0.044020,0.762353,0.167975,0.037838,0.158279,0.073814,0.185710
3,4224,18,0.111243,0.201012,0.000000,0.000000,0.107940,0.000000,0.213204,0.031273,0.434872,0.000000,0.378090,0.132351,0.007538,0.009139,0.000000,0.048160,0.111620,0.070410,0.741992,0.700006,0.717448,0.748885,0.781588,0.006910,0.839688,0.864646,0.792512,0.775334,0.070134,0.638341,0.227013,0.742325,0.736422,0.765292,0.597874,0.035716,0.634286,"{""type"":""Polygon"",""coordinates"":[[[33.08144648...",0.166209,0.000000,0.065305,0.096765,0.000000,0.054840,0.069904,0.001322,0.180683,0.681923,0.614671,0.062221,0.217482,0.400842,0.000000,0.172677,0.047671,0.801026

In [8]:
train.columns

Index(['id', 'area', 'nd_mean_2021-04-16', 'nd_mean_2021-04-19',
       'nd_mean_2021-04-22', 'nd_mean_2021-04-26', 'nd_mean_2021-04-28',
       'nd_mean_2021-05-02', 'nd_mean_2021-05-04', 'nd_mean_2021-05-07',
       'nd_mean_2021-05-16', 'nd_mean_2021-05-17', 'nd_mean_2021-05-21',
       'nd_mean_2021-05-26', 'nd_mean_2021-05-27', 'nd_mean_2021-05-29',
       'nd_mean_2021-06-06', 'nd_mean_2021-06-09', 'nd_mean_2021-06-13',
       'nd_mean_2021-06-16', 'nd_mean_2021-06-18', 'nd_mean_2021-06-19',
       'nd_mean_2021-06-20', 'nd_mean_2021-06-22', 'nd_mean_2021-06-25',
       'nd_mean_2021-06-27', 'nd_mean_2021-07-07', 'nd_mean_2021-07-08',
       'nd_mean_2021-07-09', 'nd_mean_2021-07-13', 'nd_mean_2021-07-15',
       'nd_mean_2021-07-17', 'nd_mean_2021-07-20', 'nd_mean_2021-07-26',
       'nd_mean_2021-07-27', 'nd_mean_2021-07-29', 'nd_mean_2021-07-31',
       'nd_mean_2021-08-07', 'nd_mean_2021-08-12', '.geo',
       'nd_mean_2021-04-15', 'nd_mean_2021-04-18', 'nd_mean_2021-04-25',


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4830 entries, 0 to 4829
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  4830 non-null   int64  
 1   area                4830 non-null   int64  
 2   nd_mean_2021-04-16  4830 non-null   float64
 3   nd_mean_2021-04-19  4830 non-null   float64
 4   nd_mean_2021-04-22  4830 non-null   float64
 5   nd_mean_2021-04-26  4830 non-null   float64
 6   nd_mean_2021-04-28  4830 non-null   float64
 7   nd_mean_2021-05-02  4830 non-null   float64
 8   nd_mean_2021-05-04  4830 non-null   float64
 9   nd_mean_2021-05-07  4830 non-null   float64
 10  nd_mean_2021-05-16  4830 non-null   float64
 11  nd_mean_2021-05-17  4830 non-null   float64
 12  nd_mean_2021-05-21  4830 non-null   float64
 13  nd_mean_2021-05-26  4830 non-null   float64
 14  nd_mean_2021-05-27  4830 non-null   float64
 15  nd_mean_2021-05-29  4830 non-null   float64
 16  nd_mea

In [10]:
columns_to_train=['nd_mean_2021-04-16', 'nd_mean_2021-04-19',
       'nd_mean_2021-04-22', 'nd_mean_2021-04-26', 'nd_mean_2021-04-28',
       'nd_mean_2021-05-02', 'nd_mean_2021-05-04', 'nd_mean_2021-05-07',
       'nd_mean_2021-05-16', 'nd_mean_2021-05-17', 'nd_mean_2021-05-21',
       'nd_mean_2021-05-26', 'nd_mean_2021-05-27', 'nd_mean_2021-05-29',
       'nd_mean_2021-06-06', 'nd_mean_2021-06-09', 'nd_mean_2021-06-13',
       'nd_mean_2021-06-16', 'nd_mean_2021-06-18', 'nd_mean_2021-06-19',
       'nd_mean_2021-06-20', 'nd_mean_2021-06-22', 'nd_mean_2021-06-25',
       'nd_mean_2021-06-27', 'nd_mean_2021-07-07', 'nd_mean_2021-07-08',
       'nd_mean_2021-07-09', 'nd_mean_2021-07-13', 'nd_mean_2021-07-15',
       'nd_mean_2021-07-17', 'nd_mean_2021-07-20', 'nd_mean_2021-07-26',
       'nd_mean_2021-07-27', 'nd_mean_2021-07-29', 'nd_mean_2021-07-31',
       'nd_mean_2021-08-07', 'nd_mean_2021-08-12', 
       'nd_mean_2021-04-15', 'nd_mean_2021-04-18', 'nd_mean_2021-04-25',
       'nd_mean_2021-04-27', 'nd_mean_2021-04-29', 'nd_mean_2021-04-30',
       'nd_mean_2021-06-07', 'nd_mean_2021-06-12', 'nd_mean_2021-06-28',
       'nd_mean_2021-08-01', 'nd_mean_2021-08-11', 'nd_mean_2021-05-01',
       'nd_mean_2021-05-03', 'nd_mean_2021-05-10', 'nd_mean_2021-05-15',
       'nd_mean_2021-06-02', 'nd_mean_2021-06-03', 'nd_mean_2021-07-04',
       'nd_mean_2021-08-10', 'nd_mean_2021-08-23', 'nd_mean_2021-04-20',
       'nd_mean_2021-04-23', 'nd_mean_2021-05-09', 'nd_mean_2021-05-19',
       'nd_mean_2021-05-20', 'nd_mean_2021-06-04', 'nd_mean_2021-06-05',
       'nd_mean_2021-06-10', 'nd_mean_2021-07-05', 'nd_mean_2021-08-13',
       'nd_mean_2021-08-27', 'nd_mean_2021-05-08', 'nd_mean_2021-05-24',
]

In [11]:
# выбираем поля для нейросети СatBoost
# Общий трейн (для дальнейшего прогноза)
X_train_all=train[columns_to_train]
y_train_all=train[['crop']]

testnet=test[columns_to_train]

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4830 entries, 0 to 4829
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  4830 non-null   int64  
 1   area                4830 non-null   int64  
 2   nd_mean_2021-04-16  4830 non-null   float64
 3   nd_mean_2021-04-19  4830 non-null   float64
 4   nd_mean_2021-04-22  4830 non-null   float64
 5   nd_mean_2021-04-26  4830 non-null   float64
 6   nd_mean_2021-04-28  4830 non-null   float64
 7   nd_mean_2021-05-02  4830 non-null   float64
 8   nd_mean_2021-05-04  4830 non-null   float64
 9   nd_mean_2021-05-07  4830 non-null   float64
 10  nd_mean_2021-05-16  4830 non-null   float64
 11  nd_mean_2021-05-17  4830 non-null   float64
 12  nd_mean_2021-05-21  4830 non-null   float64
 13  nd_mean_2021-05-26  4830 non-null   float64
 14  nd_mean_2021-05-27  4830 non-null   float64
 15  nd_mean_2021-05-29  4830 non-null   float64
 16  nd_mea

In [14]:
# Разобьем трейн на 5 частей и сделаем 5 подвыборок для прогнозирования с переобучением
X_train_1=train.iloc[:950]
X_train_2=train.iloc[950:1800]
X_train_3=train.iloc[1800:2750]
X_train_4=train.iloc[2750:3700]
X_train_5=train.iloc[3700:]

X_train_podvib_1=pd.concat([X_train_2,X_train_3,X_train_4,X_train_5])
X_train_podvib_2=pd.concat([X_train_1,X_train_3,X_train_4,X_train_5])
X_train_podvib_3=pd.concat([X_train_1,X_train_2,X_train_4,X_train_5])
X_train_podvib_4=pd.concat([X_train_1,X_train_2,X_train_3,X_train_5])
X_train_podvib_5=pd.concat([X_train_1,X_train_2,X_train_3,X_train_4])

y_train_podvib_1=X_train_podvib_1[['crop']]
X_train_podvib_1=X_train_podvib_1[columns_to_train]

y_train_podvib_2=X_train_podvib_2[['crop']]
X_train_podvib_2=X_train_podvib_2[columns_to_train]

y_train_podvib_3=X_train_podvib_3[['crop']]
X_train_podvib_3=X_train_podvib_3[columns_to_train]

y_train_podvib_4=X_train_podvib_4[['crop']]
X_train_podvib_4=X_train_podvib_4[columns_to_train]

y_train_podvib_5=X_train_podvib_5[['crop']]
X_train_podvib_5=X_train_podvib_5[columns_to_train]


In [45]:
# Определяем веса для классов кластеров для подвыборки 1
classes = np.unique(y_train_podvib_1)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train_podvib_3), y = np.ravel(y_train_podvib_3))
class_weights = dict(zip(classes, weights))
class_weights

{0: 0.9458800585080449,
 1: 1.000515729757607,
 2: 0.9915665729619217,
 3: 1.0399356740820156,
 4: 0.9707280460345259,
 5: 1.0341151385927505,
 6: 1.0245576973857935}

In [46]:
#X_train,X_test,y_train,y_test = train_test_split(X_train_all, 
 #                                                y_train_all, 
#                                                 test_size=0.2, 
#                                                 random_state=29)


In [ ]:
#Обучаем 
pool_train_solution = Pool(X_train_podvib_3, y_train_podvib_3)
#pool_test_solution = Pool(X_test,y_test)
testnet_solution = Pool(testnet)

model_test_1 = CatBoostClassifier(iterations=1000,  task_type='CPU',depth=8, verbose =True, random_seed=9, learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'MVS',objective='MultiClass',eval_metric = 'Accuracy',loss_function='MultiClass', class_weights=class_weights)
model_test_1.fit(pool_train_solution,verbose=True)
#model_test.fit(pool_train_solution,eval_set=pool_test_solution,verbose=True,early_stopping_rounds=50 )



0:	learn: 0.7579445	total: 934ms	remaining: 15m 32s
1:	learn: 0.7958461	total: 1.91s	remaining: 15m 55s
2:	learn: 0.8068167	total: 2.89s	remaining: 15m 59s
3:	learn: 0.8427637	total: 3.9s	remaining: 16m 10s
4:	learn: 0.8485753	total: 4.87s	remaining: 16m 9s
5:	learn: 0.8625256	total: 5.86s	remaining: 16m 10s
6:	learn: 0.8778889	total: 6.83s	remaining: 16m 8s
7:	learn: 0.8754634	total: 7.8s	remaining: 16m 6s
8:	learn: 0.8822384	total: 8.84s	remaining: 16m 13s
9:	learn: 0.8899512	total: 9.86s	remaining: 16m 16s
10:	learn: 0.8884691	total: 10.8s	remaining: 16m 14s
11:	learn: 0.8857883	total: 11.8s	remaining: 16m 15s
12:	learn: 0.8952331	total: 12.8s	remaining: 16m 13s
13:	learn: 0.8930373	total: 13.8s	remaining: 16m 13s
14:	learn: 0.8941882	total: 14.8s	remaining: 16m 13s
15:	learn: 0.8983871	total: 15.8s	remaining: 16m 12s
16:	learn: 0.9013002	total: 16.8s	remaining: 16m 11s
17:	learn: 0.8944590	total: 17.8s	remaining: 16m 9s
18:	learn: 0.8951619	total: 18.8s	remaining: 16m 8s
19:	learn:

154:	learn: 0.9582352	total: 2m 33s	remaining: 13m 58s
155:	learn: 0.9585131	total: 2m 34s	remaining: 13m 56s
156:	learn: 0.9582576	total: 2m 35s	remaining: 13m 55s
157:	learn: 0.9582576	total: 2m 36s	remaining: 13m 54s
158:	learn: 0.9582437	total: 2m 37s	remaining: 13m 54s
159:	learn: 0.9582437	total: 2m 38s	remaining: 13m 53s
160:	learn: 0.9585187	total: 2m 39s	remaining: 13m 52s
161:	learn: 0.9580184	total: 2m 40s	remaining: 13m 51s
162:	learn: 0.9577682	total: 2m 41s	remaining: 13m 50s
163:	learn: 0.9580120	total: 2m 42s	remaining: 13m 49s
164:	learn: 0.9582557	total: 2m 43s	remaining: 13m 48s
165:	learn: 0.9582557	total: 2m 44s	remaining: 13m 47s
166:	learn: 0.9585113	total: 2m 45s	remaining: 13m 45s
167:	learn: 0.9587433	total: 2m 46s	remaining: 13m 44s
168:	learn: 0.9589989	total: 2m 47s	remaining: 13m 44s
169:	learn: 0.9595171	total: 2m 48s	remaining: 13m 43s
170:	learn: 0.9592797	total: 2m 49s	remaining: 13m 42s
171:	learn: 0.9590241	total: 2m 50s	remaining: 13m 41s
172:	learn

305:	learn: 0.9719685	total: 5m 4s	remaining: 11m 29s
306:	learn: 0.9719685	total: 5m 5s	remaining: 11m 28s
307:	learn: 0.9722350	total: 5m 6s	remaining: 11m 27s
308:	learn: 0.9719685	total: 5m 7s	remaining: 11m 26s
309:	learn: 0.9722350	total: 5m 8s	remaining: 11m 25s
310:	learn: 0.9727429	total: 5m 9s	remaining: 11m 24s
311:	learn: 0.9730069	total: 5m 9s	remaining: 11m 23s
312:	learn: 0.9735212	total: 5m 10s	remaining: 11m 22s
313:	learn: 0.9732571	total: 5m 11s	remaining: 11m 21s
314:	learn: 0.9737714	total: 5m 13s	remaining: 11m 20s
315:	learn: 0.9737852	total: 5m 14s	remaining: 11m 19s
316:	learn: 0.9740354	total: 5m 15s	remaining: 11m 18s
317:	learn: 0.9737714	total: 5m 16s	remaining: 11m 17s
318:	learn: 0.9737714	total: 5m 17s	remaining: 11m 16s
319:	learn: 0.9740354	total: 5m 18s	remaining: 11m 15s
320:	learn: 0.9740354	total: 5m 19s	remaining: 11m 14s
321:	learn: 0.9740354	total: 5m 20s	remaining: 11m 13s
322:	learn: 0.9740354	total: 5m 21s	remaining: 11m 12s
323:	learn: 0.974

In [ ]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test_1.predict(X_train_all)
train['Podvib_3']=y_pred
train['Podvib_3'].value_counts()

In [ ]:
print("Значение метрики recall на трейне : ", recall_score(y_train_all, y_pred, average='macro', zero_division=0))

In [ ]:
# Значимость признаков CatBoost в модели прогностики

def plot_feature_importance(importance,names,model_type):
    
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    #Define size of bar plot
    plt.figure(figsize=(10,40))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

#plot the catboost result
plot_feature_importance(model_test_1.get_feature_importance(),X_train_podvib_3.columns,'CATBOOST')

In [ ]:
#Получение ответов
y_pred_solution = model_test_1.predict(testnet_solution)

In [ ]:
# Заводим результаты в тест
test['Podvib_3']=y_pred_solution
test['Podvib_3'].value_counts()

In [ ]:
# После выполнения предварительного прогона по 5-и сидам 15,39,56,77,98
# сохраням прогноз на трейне и тесте для дальнейшей обработки
#train_prognoz=train[['id','crop','Podvib_1','Podvib_2','Podvib_3','Podvib_4','Podvib_5']]
#test_prognoz=test[['id','Podvib_1','Podvib_2','Podvib_3','Podvib_4','Podvib_5']]
#train_prognoz.to_csv('train_prognoz.csv', sep=',', index=None)
#test_prognoz.to_csv('test_prognoz.csv', sep=',', index=None)

In [ ]:
#train_chance=train_prognoz.loc[(train_prognoz['crop']!=train_prognoz['Podvib_1'])|(train_prognoz['Podvib_1]!=train_prognoz['Podvib_2'])|(train_prognoz['Podvib_2']!=train_prognoz['Podvib_3'])|(train_prognoz['Podvib_3']!=train_prognoz['Podvib_4'])|(train_prognoz['Podvib_4']!=train_prognoz['Podvib_5'])]
#train_chance

In [ ]:
#train_chance['crop'].value_counts()

In [ ]:
#test_chance=test_prognoz.loc[(test_prognoz['Podvib_1']!=test_prognoz['Podvib_2'])|(test_prognoz['Podvib_2']!=test_prognoz['Podvib_3'])|(test_prognoz['Podvib_3']!=test_prognoz['Podvib_4'])|(test_prognoz['Podvib_4']!=test_prognoz['Podvib_5'])]
#test_chance

In [ ]:
prognoz_itog=test[['id','Podvib_3']]
prognoz_itog.columns=['id','crop']
prognoz_itog

In [ ]:
prognoz_itog.to_csv('Казань подвыборка_3 проверка.csv', sep=',', index=None)